<font size = 6><b>구글 sentencepiece 활용하여 워드피스 단위 사전 만들기 </b></font>

In [30]:
import sentencepiece as spm
from tqdm import tqdm_notebook, tqdm
import pandas as pd

In [11]:
## 학습 config 설정 ##학습 데이터는 korquad 관련 데이터 다운받기
train ="""--input=./data/ratings_train.txt \
    --model_prefix=sentpiece \
    --vocab_size=30000 \
    --model_type=bpe --character_coverage=0.9995"""
## model_prefix : model 이름, model이름.model , model이름.vocab 생성됨
## vocab_size : 사전 갯수
## character_coverage : 모델에 의해 커버되는 char의 amount
## model_type : unigram(default), bpe, char, word (if using word,must be pretokenizing)
# BPE : Byte Pair Encoding : 모든 단어들을 character, unicode 단위로 변경 후 가장 많이 등장하는 집합의 유니그램을 합치는 것을 반복하여 사전으로  만듬 

In [12]:
spm.SentencePieceTrainer.Train(train)

In [31]:
sp = spm.SentencePieceProcessor()

In [32]:
sp.Load('sentpiece.model')

True

In [33]:
print(sp.EncodeAsPieces('이 영화는 정말 기분나쁘고 소름돋아서 다시는 보기 싫었어요.')) #spm으로 만들어진 사전 기반 워드피스 토크나이징 확인

['▁이', '▁영화는', '▁정말', '▁기분나쁘', '고', '▁소름돋', '아서', '▁다시는', '▁보기', '▁싫', '었어요', '.']


In [18]:
vocab = pd.read_csv("sentpiece.vocab", sep ="delimeter", header=None, encoding = 'UTF8')# 사전 파일 불러오기
len(vocab)

C:\Users\leevi\Anaconda3\Lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


30000

In [19]:
### 사전 단어만 리스트에 넣기
vocab_list = []
for i in vocab.values:
    for v in i:
        word = v.split("\t")[0]
        vocab_list.append(word)
len(vocab_list)      

30000

In [20]:
vocab_list[:10]

['<unk>', '<s>', '</s>', '▁1', '▁0', '..', '영화', '▁영화', '▁이', '▁9']

In [21]:
### "_" 제거 후 중복 단어 처리
n_vocab_list = []
for v in vocab_list:
    n_v = v.replace("▁","")
    if n_v not in n_vocab_list:
        n_vocab_list.append(n_v)
len(n_vocab_list)

24003

In [22]:
n_vocab_list[:10]

['<unk>', '<s>', '</s>', '1', '0', '..', '영화', '이', '9', '아']

<font size = 6><b>konlpy 활용하여 사전에 명사 추가 및 "명사 + 조사" 형태 분리 및 중복 삭제 작업 </b></font>

In [23]:
from konlpy.tag import Okt 

In [24]:
okt = Okt()

In [25]:
print(okt.pos("안녕하세요 트위터 형태소 분석기 테스트해볼게요"))

[('안녕하세요', 'Adjective'), ('트위터', 'Noun'), ('형태소', 'Noun'), ('분석', 'Noun'), ('기', 'Noun'), ('테스트', 'Noun'), ('해볼게요', 'Verb')]


In [26]:
vocabs = pd.DataFrame(data =n_vocab_list, columns=["word"],index = None)

In [47]:
vocabs.head(4)

,word,nouns,removal_nouns
0,<unk>,,<unk>
1,<s>,,<s>
2,</s>,,</s>
3,1,,1


In [28]:
## 명사 + 조사 분리 목적 / 중복 제거를 통해 사전 크기 줄이기
nouns = []
removal_nouns = []
for i in vocabs["word"]:
    extract_nouns = okt.nouns(i)
    if len(extract_nouns) == 1:
        nouns.append("".join(extract_nouns))           # 김일훈에게 -> 김일훈
        removal = i.replace("".join(extract_nouns),"") # 김일훈에게 -> 에게
        removal_nouns.append(removal)
        
    else:
        nouns.append("")
        removal_nouns.append(i)   

In [29]:
vocabs["nouns"] = nouns
vocabs["removal_nouns"] = removal_nouns

In [30]:
final_vocab = []

In [31]:
for i in vocabs["nouns"]:
    if i not in final_vocab:
        final_vocab.append(i)

In [32]:
for i in vocabs["removal_nouns"]:
    if i not in final_vocab:
        final_vocab.append(i)

In [33]:
len(final_vocab) # 명사, 조사 분리 및 중복제거 된 사전

18615

<font size = 6><b>사전 모든 단어에 "\_" 추가하여 의미 구분 작업 및 실사용 "\_"만 붙이기 </b></font>

In [34]:
### "_" 사전 모든단어에 붙여넣기
u_final_vocab=[]
for i in final_vocab:
    u = f'{i}_'
    u_final_vocab.append(u)
    u_final_vocab.append(i)  

In [35]:
len(u_final_vocab)

37230

In [36]:
## 일단 사전 파일로 만들기
bert_specific=["[UNK]","[CLS]","[SEP]","[MASK]"]
for i in bert_specific:
    u_final_vocab.append(i)

In [37]:
fin_vocab = pd.DataFrame(data = u_final_vocab, columns=None, index =None)

In [38]:
fin_vocab.to_csv("./test_vocab.txt", header=None, index=None)

In [4]:
import tokenization_kor as tokenization

C:\Users\leevi\Anaconda3\Lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\leevi\Anaconda3\Lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\leevi\Anaconda3\Lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\leevi\Anaconda3\Lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [5]:
tokenizer = tokenization.FullTokenizer("test_vocab.txt")

W0602 09:39:19.016005 16896 deprecation_wrapper.py:119] From C:\coding\git\BertStudy\tokenization_kor.py:126: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [6]:
tokenizer.tokenize("새로 만든 사전을 확인 좀 해보겠습니다")

['새로_', '만든_', '사전', '을_', '확인_', '좀_', '해보', '겠습니다']

In [7]:
data = pd.read_csv("./data/ratings_train.txt",sep="\t")

In [8]:
data.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [10]:
#### "_" 붙인 사전단어 중에서 실제로 쓰이는 것만 남기기 (불필요한 사전 단어 제거)
use_vocab = []
for sentence in tqdm(data["document"]):
    try:
        token = tokenizer.tokenize(sentence)
        for t in token:
            if t not in use_vocab:
                use_vocab.append(t)
    except:
        pass

100%|█████████████████████████████████████████████████████████████████████████| 150000/150000 [05:02<00:00, 496.60it/s]


In [11]:
len(use_vocab)

26380

In [16]:
bert_specific=["[UNK]","[CLS]","[SEP]","[MASK]"]
for i in bert_specific:
    if i not in use_vocab:
        use_vocab.append(i)
len(use_vocab)    

26383

In [17]:
final_vocab = pd.DataFrame(data = use_vocab, index =None)

In [22]:
final_vocab.to_csv("nsmc_vocab.txt", index=None, header=None)## 최종 nsmc용 사전 완성

In [23]:
tokenizer = tokenization.FullTokenizer("nsmc_vocab.txt")

In [34]:
print(sp.EncodeAsPieces('이 영화는 정말 기분나쁘고 소름돋아서 다시는 보기 싫었어요.')) #spm으로 만들어진 사전 기반 워드피스 토크나이징 확인
print(tokenizer.tokenize('이 영화는 정말 기분나쁘고 소름돋아서 다시는 보기 싫었어요.')) #완성된 사전 기반 워드피스  토크나이징 비교

['▁이', '▁영화는', '▁정말', '▁기분나쁘', '고', '▁소름돋', '아서', '▁다시는', '▁보기', '▁싫', '었어요', '.']
['이_', '영화', '는_', '정말_', '기분', '나쁘', '고_', '소름', '돋', '아서_', '다시는_', '보기_', '싫', '었어', '요', '.']
